In [1]:
import queue

class Intcode:
    def __init__(self, intcode=None, inputs=None, outputs=None, manual=True):
        if intcode == None:
            self.intcode = [99]
        self.inputs = inputs
        if inputs == None:
            self.inputs = queue.Queue()
        if type(inputs) == list:
            q = inputs
            self.inputs = queue.Queue()
            [self.inputs.put(i) for i in q]
        self.outputs = outputs
        if outputs == None:
            self.outputs = queue.Queue()
        if type(outputs) == list:
            q = outputs
            self.outputs = queue.Queue()
            [self.outputs.put(i) for i in q]
        self.base = 0
        self.pc = 0
        self.mem = intcode.copy()
        self.manual = manual
        
    def __mem(self, p, v=None):
        if p >= len(self.mem):
            self.mem += [0] * (1 + p - len(self.mem))
        if not v == None:
            self.mem[p] = v
        return self.mem[p]

    def __get(self, var, mode):
        tar = self.pc + ['A', 'B', 'C'].index(var) + 1
        if mode == 0:
            return self.__mem(self.__mem(tar))
        elif mode == 1:
            return self.__mem(tar)
        elif mode == 2:
            return self.__mem(self.base + self.__mem(tar))
        return None

    def __set(self, var, mode, value):
        tar = self.pc + ['A', 'B', 'C'].index(var) + 1
        if mode == 0:
            return self.__mem(self.__mem(tar), value)
        elif mode == 1:
            return self.__mem(tar, value)
        elif mode == 2:
            return self.__mem(self.base + self.__mem(tar), value)
        return None

    def run(self):
        while not (self.__mem(self.pc) % 100 == 99):
            op = self.__mem(self.pc) % 100
            modes = []
            for i in range(3):
                modes += [int(self.__mem(self.pc) / 100 / 10**i) % 10]
            
            l = 4
            
            if op == 1:
                A = self.__get('A', modes[0])
                B = self.__get('B', modes[1])
                self.__set('C', modes[2], A + B)
            elif op == 2:
                A = self.__get('A', modes[0])
                B = self.__get('B', modes[1])
                self.__set('C', modes[2], A * B)
            elif op == 3:
                if self.manual:
                    inp = input('Gief pls: ')
                    if inp == 'abort':
                        print("Aborted")
                        return "Aborted"
                    self.inputs.put(int(inp))
                else:
                    self.outputs.put("request")
                self.__set('A', modes[0], self.inputs.get())
                self.inputs.task_done()
                l = 2
            elif op == 4:
                A = self.__get('A', modes[0])
                if self.manual:
                    print('Take pls:', A)
                self.outputs.put(A)
                l = 2
            elif op == 5:
                A = self.__get('A', modes[0])
                B = self.__get('B', modes[1])
                if A:
                    self.pc = B
                    l = 0
                else:
                    l = 3
            elif op == 6:
                A = self.__get('A', modes[0])
                B = self.__get('B', modes[1])
                if not A:
                    self.pc = B
                    l = 0
                else:
                    l = 3
            elif op == 7:
                A = self.__get('A', modes[0])
                B = self.__get('B', modes[1])
                if A < B:
                    self.__set('C', modes[2], 1)
                else:
                    self.__set('C', modes[2], 0)
            elif op == 8:
                A = self.__get('A', modes[0])
                B = self.__get('B', modes[1])
                if A == B:
                    self.__set('C', modes[2], 1)
                else:
                    self.__set('C', modes[2], 0)
            elif op == 9:
                A = self.__get('A', modes[0])
                self.base += A
                l = 2
            else:
                return "Something went wrong."
            self.pc += l
        return self.mem, list(self.outputs.queue)

def process_Intcode_test(params, output, message):
    prg = Intcode(*params)
    if not prg.run() == output:
        print(message)
        return False
    return True

process_Intcode_test([[1,9,10,3,2,3,11,0,99,30,40,50]],
                     ([3500,9,10,70,2,3,11,0,99,30,40,50], []), "FAIL: process_Intcode_test 01")
process_Intcode_test([[1,0,0,0,99]], ([2,0,0,0,99], []), "FAIL: process_Intcode_test 02")
process_Intcode_test([[2,3,0,3,99]], ([2,3,0,6,99], []), "FAIL: process_Intcode_test 03")
process_Intcode_test([[2,4,4,5,99,0]], ([2,4,4,5,99,9801], []), "FAIL: process_Intcode_test 04")
process_Intcode_test([[1,1,1,4,99,5,6,0,99]], ([30,1,1,4,2,5,6,0,99], []), "FAIL: process_Intcode_test 05")
process_Intcode_test([[1002,4,3,4,33]], ([1002,4,3,4,99], []), "FAIL: process_Intcode_test 07")
process_Intcode_test([[1101,100,-1,4,0]], ([1101,100,-1,4,99], []), "FAIL: process_Intcode_test 08")
#testing with inputs
process_Intcode_test([[3,0,4,0,99], [456], [], False], ([456,0,4,0,99], [456]), "FAIL: process_Intcode_test 06")
process_Intcode_test([[3,9,8,9,10,9,4,9,99,-1,8], [8], [], False], ([3,9,8,9,10,9,4,9,99,1,8], [1]), "FAIL: process_Intcode_test 09")
process_Intcode_test([[3,9,8,9,10,9,4,9,99,-1,8], [5], [], False], ([3,9,8,9,10,9,4,9,99,0,8], [0]), "FAIL: process_Intcode_test 10")
process_Intcode_test([[3,9,7,9,10,9,4,9,99,-1,8], [5], [], False], ([3,9,7,9,10,9,4,9,99,1,8], [1]), "FAIL: process_Intcode_test 11")
process_Intcode_test([[3,9,7,9,10,9,4,9,99,-1,8], [8], [], False], ([3,9,7,9,10,9,4,9,99,0,8], [0]), "FAIL: process_Intcode_test 12")
process_Intcode_test([[3,3,1108,-1,8,3,4,3,99], [8], [], False], ([3,3,1108,1,8,3,4,3,99], [1]), "FAIL: process_Intcode_test 13")
process_Intcode_test([[3,3,1108,-1,8,3,4,3,99], [5], [], False], ([3,3,1108,0,8,3,4,3,99], [0]), "FAIL: process_Intcode_test 14")
process_Intcode_test([[3,3,1107,-1,8,3,4,3,99], [5], [], False], ([3,3,1107,1,8,3,4,3,99], [1]), "FAIL: process_Intcode_test 15")
process_Intcode_test([[3,3,1107,-1,8,3,4,3,99], [8], [], False], ([3,3,1107,0,8,3,4,3,99], [0]), "FAIL: process_Intcode_test 16")
#day 9 tests
process_Intcode_test([[109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99], [], [], False], ([109, 1, 204, -1, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 1], [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]), "FAIL: process_Intcode_test 17")
process_Intcode_test([[1102,34915192,34915192,7,4,7,99,0], [], [], False], ([1102, 34915192, 34915192, 7, 4, 7, 99, 1219070632396864], [1219070632396864]), "FAIL: process_Intcode_test 18")
process_Intcode_test([[104,1125899906842624,99], [], [], False], ([104, 1125899906842624, 99], [1125899906842624]), "FAIL: process_Intcode_test 18")

FAIL: process_Intcode_test 06
FAIL: process_Intcode_test 09
FAIL: process_Intcode_test 10
FAIL: process_Intcode_test 11
FAIL: process_Intcode_test 12
FAIL: process_Intcode_test 13
FAIL: process_Intcode_test 14
FAIL: process_Intcode_test 15
FAIL: process_Intcode_test 16


True

In [2]:
with open("input13.txt") as infile:
    intcode = [int(i) for i in infile.readlines()[0].split(',')]
    inp = queue.Queue()
    out = queue.Queue()
    prg = Intcode(intcode, inputs=inp, outputs=out, manual=False)
    
out = prg.run()[1]
screen = []
for i in range(0, len(out), 3):
    (x, y, z) = (out[i], out[i+1], out[i+2])
    if y >= len(screen):
        screen += [[]] * (1 + y - len(screen))
    row = screen[y]
    if x >= len(row):
        row += [''] * (1 + x - len(row))
    row[x] = z
    
block = [' ', '#', 'x' ,'=' ,'o']
    
screen_img = '\n'.join([''.join(block[i] for i in row) for row in screen])
screen_img.count('x')

260

In [3]:
from IPython.display import clear_output

def update(screen, x, y, z):
    if y >= len(screen):
        screen += [[]] * (1 + y - len(screen))
    row = screen[y]
    if x >= len(row):
        row += [''] * (1 + x - len(row))
    row[x] = z

    return screen

with open("input13.txt") as infile:
    intcode = [int(i) for i in infile.readlines()[0].split(',')]
    intcode[0] = 2
    inp = queue.Queue()
    out = queue.Queue()
    prg = Intcode(intcode, inputs=inp, outputs=out, manual=False)

import threading

t = threading.Thread(target=Intcode.run, name='arcade', args=(prg,))
t.start()

screen = []
score = 0

while t.is_alive() or not out.empty():
    x = out.get()
    if x == "request":
        clear_output(wait=True)
        print('\n'.join([''.join(block[i] for i in row) for row in screen]))
        print("Score: ", score)
        
        target = -1
        pos = -1
        for row in screen:
            if 4 in row:
                target = row.index(4)
            if 3 in row:
                pos = row.index(3)
        if target > pos:
            inp.put(1)
        elif target < pos:
            inp.put(-1)
        else:
            inp.put(0)
        continue
    y = out.get()
    z = out.get()
    if (x, y) == (-1, 0):
        score = z
    else:
        update(screen, x, y, z)

clear_output(wait=True)
print('\n'.join([''.join(block[i] for i in row) for row in screen]))
print("Score: ", score)

###################################
#                                 #
#                                 #
#                                 #
#                                 #
#                                 #
#                  o              #
#                                 #
#                                 #
#                                 #
#                                 #
#                                 #
#                                 #
#                                 #
#                                 #
#                                 #
#                                 #
#                                 #
#                                 #
#                                 #
#                                 #
#                  =              #
#                                 #
Score:  12952
